# TYPING IN PYTHON
[视频链接](https://www.bilibili.com/video/BV1Hp4y1M7gq)

## 类型注解放在什么地方
变量后面或者函数后面
### 变量类型
- int
- float
- str
- bool
- bytes
- Any
- Tuple
- List, list[dtype]
- Dist, dict[type(key),type(value)]

既可以是Python的基础类型，也可以是未实例化的类或者是typing里面提供的泛型

In [ ]:
def f(a: int, b: int) -> int:
    return a + b

## 使用自己的类进行注解
### 注意循环引入的问题
解决方式：
- TYPE_CHECKING: 有时候没啥用
```python
if TYPE_CHECKING:
    from student import student
```
- 在一个注解的类型名称外面加''
```python
def get_student_name(self, s:'Student')->str:
    return s.get_student_name()
```
- import annotations 仓库
其实是自动在循环引用的类型外部加引号
```python
from __future__ import annotations
```

In [6]:
from __future__ import annotations


class Teacher:
    def get_name(self) -> str:
        return "sch"

    # 出现了循环引入的问题
    # 1. 在一个类的外面加引号
    # def get_student_name(self, s: 'Student') -> str:
    def get_student_name(self, s: Student) -> str:
        return s.get_name()


class Student:
    def get_name(self) -> str:
        return "zhaoyue"

    # 标注自定义类帮助获取自定义类中函数所标注的typing信息
    def get_teacher_name(self, teacher: Teacher) -> str:
        return teacher.get_name()

In [3]:
s = Student()
t = Teacher()

print(s.get_name())
print(s.get_teacher_name(t))

zhaoyue
sch


## final不能修改的注解器


python中没有常量这一概念，但可以限定某些变量不能被修改

Final: 设置变量不能被修改
@final: 装饰器，设置父类 函数不能被子类函数覆盖

In [2]:
from typing import Final, final

a: Final[int] = 10


class Human:

    @final
    def get_name(self) -> str:
        return ""

11


## 类型注解选项
- Optional：在变量初始化的时候
Optional[Teacher]: 既可以是Teacher， 也可以是None
也可以：Teacher | None
- Union: 参数中
Union[str, int]: 函数参数既可以是str也可以是int
也可以：str | int

更精确的定义函数的类型 

## 特殊类型
- Self：返回类自身,return this 的情形
有助于链式调用

- ClassVar：相当于static

In [33]:
from __future__ import annotations
from typing_extensions import Self
from typing import ClassVar


class Number:

    op_num: ClassVar[int] = 0

    def __init__(self, number: int) -> None:
        self._number = number

    # 将对象本身返回出去
    # also: def double(self)->'Number': but cause trouble when changing class name
    def double(self) -> Self:
        self._number *= 2
        Number.op_num += 1

        return self

    def add_one(self) -> Self:
        self._number += 1
        Number.op_num += 1
        return self

    def __str__(self) -> str:
        return f"Current number is {self._number}"


n: Number = Number(10)
n1: Number = Number(13)

print(n.double().add_one())
print(Number.op_num)
print(n.op_num)
print(n1.op_num)

Current number is 21


AttributeError: type object 'Number' has no attribute '__op_num'

## Literal 
限制变量只能是指定的字面量
Literal可以接受任何不可变的类型，如整数，浮点数，字符串，布尔值等
常常用在```类型检查```和```自动补全```中

In [ ]:
from typing import Literal


def process_data(data_type: Literal['csv', 'json']) -> None:
    match data_type:
        case "csv":
            pass
        case "json":
            pass
        case _:
            pass

## TypeVar
是一个特殊的类型，它可以用来表示“任意类型”，他常常用在泛型中，可以帮助我们写出更加通用的函数

In [ ]:
from typing import TypeVar, Union

T = TypeVar('T', int, str)


def add(a, b):
    return a + b

# Union不能保证前后变量的类型一致
# def add(a: Union[int,str],b:Union[int, str])->Union[int,str]:
#   return a+b


# TypeVar 可以在函数中保持多个变量的一致性
def general_add(a: T, b: T) -> T:
    return a + b

## ```Generic```: 在类里面使用泛型


In [ ]:
from typing import List
from typing import Generic


T = TypeVar('T')


class MyList(Generic[T]):
    def __init__(self, items: List[T]) -> None:
        self.items: List[T] = items

    def append(self, item: T) -> None:
        self.items.append(item)

    def __str__(self) -> str:
        return str(self.items)


my_list = MyList[int]([1, 2, 3])
my_list.append(4)

print(my_list)

## ```overload```方法签名重载
对同名函数进行方法签名重载,并不会改变原来函数的意义,但是会给IDE提示,多个同样名字的函数以最后一个为准

很多函数中传入的是args,分析在内部进行,函数提示由overload来提供, 能正确运行的函数需要放在最下面

In [ ]:
from typing import overload

from sqlalchemy import over


@overload
def add(a: int, b: int) -> int:
    """_summary_

    Args:
        a (int): _description_
        b (int): _description_

    Returns:
        int: _description_
    """
    return a + b


@overload
def add(a: str, b: str) -> str:
    """_summary_

    Args:
        a (str): _description_
        b (str): _description_

    Returns:
        str: _description_
    """
    return a + b


def add(a, b):
    return a + b


add(1, 3)

## ```override```
用来检查子类是否已经正确重载了父类中的函数

In [ ]:
from typing import override


class Animal():
    def __init__(self, name: str) -> None:
        self.name = name

    def make_sound(self) -> str:
        return ""


class Dog(Animal):
    def __init__(self, name: str, breed: str) -> None:
        super().__init__(name)
        self.breed: str = breed

    @override
    def make_sound(self) -> str:
        return "Woof!"


class Cat(Animal):
    def __init__(self, name: str, color: str) -> None:
        super().__init__(name)
        self.color: str = color

    @override
    def make_sound(self) -> str:
        return "Meow!"

## ```Protocal```
是一个抽象基类,它的作用是用来定义协议,它的子类可以用来表示协议, 如```Iterable```, ```Sequence```等

In [1]:
from typing_extensions import Protocol


class Animal(Protocol):
    def make_sound(self, word: str) -> str:
        return "nothing"


class Dog:
    def make_sound(self, word: str) -> str:
        return f"dog say {word}"


class Cat:
    def make_sound(self, word: str) -> str:
        return f"cat say {word}"


def make_sound(a: Animal, word: str):
    return a.make_sound(word)


print(make_sound(Cat(), "Meow"))

cat say Meow


## TypedDict
定义字典的键和值的类型

In [3]:
from typing_extensions import TypedDict
from typing_extensions import Required, NotRequired


class Person(TypedDict):
    name: str
    age: int
    email: NotRequired[str]
    
my_Dict: Person ={'age': 14, 'name':'zy'}

## Unpack 解包
用来解包一个字典
给```**kwargs```提供更加精确的类型支持, 在写```**kwargs```的时候可以添加一个类型注解
这个类型注解是```TypedDict```类型的```Dict```, 然后通过```Unpack```可以将这个字典传递给```**kwargs```

In [4]:
from typing import Unpack
from typing_extensions import TypedDict
from typing_extensions import Required, NotRequired


class Person(TypedDict):
    name: str
    age: int
    email: NotRequired[str]
    
    
def gt_person_info(*args,**kwargs: Unpack[Person])->str:
    return f"{kwargs['name']} is {kwargs['age']} years old"


print(gt_person_info(name='zy', age=14))

zy is 14 years old
